In [4]:
!pip install dmba

   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.8 MB 487.6 kB/s eta 0:00:25
   ---------------------------------------- 0.1/11.8 MB 930.9 kB/s eta 0:00:13
    --------------------------------------- 0.2/11.8 MB 1.4 MB/s eta 0:00:09
    --------------------------------------- 0.3/11.8 MB 1.4 MB/s eta 0:00:09
   - -------------------------------------- 0.4/11.8 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.6/11.8 MB 1.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/11.8 MB 2.1 MB/s eta 0:00:06
   --- ------------------------------------ 0.9/11.8 MB 2.3 MB/s eta 0:00:05
   --- ------------------------------------ 1.1/11.8 MB 2.5 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/11.8 MB 2.5 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/11.8 MB 2.4 MB/s eta 0:00:05
   ---- -

In [6]:
#importing necessary libraries
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as sm
import matplotlib.pylab as plt
import seaborn as sns
import dmba #data mining menthods for business analytics
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection,stepwise_selection
from dmba import adjusted_r2_score, AIC_score #, BIC_score
from sklearn.metrics import mean_absolute_error, r2_score

%matplotlib inline

In [12]:
#loading the required file
used_cars = pd.read_csv("D:/Users/SHREE/Downloads/ToyotaCorolla.csv")

In [14]:
#data exploration
columns = list(used_cars.columns)
print("Variables in the data are: ")
print(columns)

# review first 5 records in the data
print("\nFirst 5 records in the data are:")
used_cars.head()

Variables in the data are: 
['Id', 'Model', 'Price', 'Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM', 'Fuel_Type', 'HP', 'Met_Color', 'Color', 'Automatic', 'CC', 'Doors', 'Cylinders', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee', 'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player', 'Central_Lock', 'Powered_Windows', 'Power_Steering', 'Radio', 'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim', 'Radio_cassette', 'Parking_Assistant', 'Tow_Bar']

First 5 records in the data are:


,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,1,0,0,0,1,0,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,0,1,0,1,0,0,0,0


In [16]:
#determining the outcome variable
used_cars.columns
outcome = 'Price'

In [18]:
#determining predictors
predictors = ['Age_08_04', 'KM', 'Fuel_Type','HP','Automatic','Doors','Quarterly_Tax','Mfr_Guarantee','Guarantee_Period','Airco','Automatic_airco','CD_Player','Powered_Windows','Sport_Model','Tow_Bar']

In [20]:
#using the predictors for further analysis
used_cars_pred=used_cars[predictors]

In [22]:
#creating dummy variables for 'Fuel_Type' predictor as it is a categorical data
used_cars_pred = pd.get_dummies(used_cars_pred, columns=['Fuel_Type'], drop_first=True)

In [24]:
#variables for the regression model
x=used_cars_pred
y = used_cars[outcome]

In [26]:
#partitioning the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.4,random_state=1)
print('Training set:', train_x.shape, 'Validation set:', valid_x.shape)

Training set: (861, 16) Validation set: (575, 16)


In [30]:
scaler = StandardScaler()

In [32]:
# Fit the scaler on the training data and testing data
train_x_scaled = scaler.fit_transform(train_x)
valid_x_scaled = scaler.transform(valid_x)

In [36]:
#fit the regression model y on x and print coefficients
used_cars_lm = LinearRegression()
used_cars_lm.fit(train_x_scaled,train_y)
print('intercept ', used_cars_lm.intercept_)
print(pd.DataFrame({'Predictor': train_x.columns, 'coefficient':used_cars_lm.coef_}))

intercept  10774.15331010453
           Predictor  coefficient
0          Age_08_04 -2087.316503
1                 KM  -734.172694
2                 HP   523.660819
3          Automatic   158.956608
4              Doors   210.131678
5      Quarterly_Tax   677.907018
6      Mfr_Guarantee    70.404503
7   Guarantee_Period   221.720284
8              Airco    41.845983
9    Automatic_airco   715.623969
10         CD_Player   143.450146
11   Powered_Windows   255.659686
12       Sport_Model   221.035490
13           Tow_Bar   -94.382525
14  Fuel_Type_Diesel   611.681603
15  Fuel_Type_Petrol   597.760444


most important car specifications for predicting the car’s price are :

Age_08_04   -2087.316503
KM           -734.172694
Automatic_airco   715.623969
Fuel_Type_Diesel   611.681603
Fuel_Type_Petrol   597.7604449

In [41]:
# predict prices in validation set, print first few predicted/actualvalues and residuals
used_cars_lm_pred = used_cars_lm.predict(valid_x_scaled)
result = pd.DataFrame({'Predicted': used_cars_lm_pred, 'Actual': valid_y,'Residual': valid_y - used_cars_lm_pred})
# print performance measures (validation set)
print("\nModel performance on validation data:")
regressionSummary(valid_y, used_cars_lm_pred)


Model performance on validation data:

Regression statistics

                      Mean Error (ME) : 66.0755
       Root Mean Squared Error (RMSE) : 1197.2318
            Mean Absolute Error (MAE) : 929.5707
          Mean Percentage Error (MPE) : -0.1101
Mean Absolute Percentage Error (MAPE) : 9.3825


In [43]:
#calculating the r-squared value for analysis
r2_all = r2_score(valid_y, used_cars_lm_pred)
print("R-squared value",r2_all)

R-squared value 0.8830182362978589


We determined on the best-set for the predictions using the below models of elimination

In [51]:
#backward elimination model
def train_model(variables):
    model = LinearRegression()
    model.fit(train_x[variables], train_y)
    return model
def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_x[variables]), model)
best_model, best_variables = backward_elimination(train_x.columns,
train_model, score_model, verbose=True)
print("Best Subset:", best_variables)

Variables: Age_08_04, KM, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, Tow_Bar, Fuel_Type_Diesel, Fuel_Type_Petrol
Start: score=14717.40
Step: score=14716.03, remove Airco
Step: score=14716.03, remove None
Best Subset: ['Age_08_04', 'KM', 'HP', 'Automatic', 'Doors', 'Quarterly_Tax', 'Mfr_Guarantee', 'Guarantee_Period', 'Automatic_airco', 'CD_Player', 'Powered_Windows', 'Sport_Model', 'Tow_Bar', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']


In [167]:
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_x[variables], train_y)
    return model
def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model,df=1)
    return AIC_score(train_y, model.predict(train_x[variables]), model)
best_model, best_variables = forward_selection(train_x.columns,
train_model, score_model, verbose=True)
print("Best Subset:", best_variables)

Variables: Age_08_04, KM, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, Tow_Bar, Fuel_Type_Diesel, Fuel_Type_Petrol
Start: score=16598.48, constant
Step: score=15340.94, add Age_08_04
Step: score=15106.55, add Automatic_airco
Step: score=15004.56, add HP
Step: score=14948.26, add KM
Step: score=14828.95, add Quarterly_Tax
Step: score=14797.06, add Powered_Windows
Step: score=14784.66, add Doors
Step: score=14766.99, add Sport_Model
Step: score=14750.16, add Guarantee_Period
Step: score=14739.27, add Automatic
Step: score=14734.58, add CD_Player
Step: score=14732.05, add Fuel_Type_Diesel
Step: score=14718.58, add Fuel_Type_Petrol
Step: score=14716.53, add Tow_Bar
Step: score=14716.03, add Mfr_Guarantee
Step: score=14716.03, add None
Best Subset: ['Age_08_04', 'Automatic_airco', 'HP', 'KM', 'Quarterly_Tax', 'Powered_Windows', 'Doors', 'Sport_Model', 'Guarantee_Period', 'Automatic', 'CD_Player', 'Fuel

In [170]:
best_model, best_variables = stepwise_selection(train_x_scaled.columns,train_model, score_model, verbose=True)
print("Best Subset:", best_variables)

Variables: Age_08_04, KM, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, Tow_Bar, Fuel_Type_Diesel, Fuel_Type_Petrol
Start: score=16598.48, constant
Step: score=15340.94, add Age_08_04
Step: score=15106.55, add Automatic_airco
Step: score=15004.56, add HP
Step: score=14948.26, add KM
Step: score=14828.95, add Quarterly_Tax
Step: score=14797.06, add Powered_Windows
Step: score=14784.66, add Doors
Step: score=14766.99, add Sport_Model
Step: score=14750.16, add Guarantee_Period
Step: score=14739.27, add Automatic
Step: score=14734.58, add CD_Player
Step: score=14732.05, add Fuel_Type_Diesel
Step: score=14718.58, add Fuel_Type_Petrol
Step: score=14716.53, add Tow_Bar
Step: score=14716.03, add Mfr_Guarantee
Step: score=14716.03, unchanged None
Best Subset: ['Age_08_04', 'Automatic_airco', 'HP', 'KM', 'Quarterly_Tax', 'Powered_Windows', 'Doors', 'Sport_Model', 'Guarantee_Period', 'Automatic', 'CD_Player',

In [61]:
# fit the model with best subset variables obtained from elimination models
outcome = 'Price'
predictors = ['Age_08_04', 'Automatic_airco', 'HP', 'KM', 
              'Quarterly_Tax', 'Powered_Windows', 'Doors', 'Sport_Model', 
              'Guarantee_Period', 'Automatic', 'CD_Player', 'Fuel_Type_Diesel', 
              'Fuel_Type_Petrol', 'Tow_Bar', 'Mfr_Guarantee']
x_train_new = train_x[predictors]
x_valid_new = valid_x[predictors]
y = train_y

# Fit the scaler on the training data and testing data
train_x_scaled_new = scaler.fit_transform(x_train_new)
valid_x_scaled_new = scaler.fit_transform(x_valid_new)

#valid_x_scaled = scaler.transform(valid_x)

# fit the regression model y on X
used_cars_lm = LinearRegression()
used_cars_lm.fit(train_x_scaled_new,y)

# regression model, print coefficients
print('intercept', used_cars_lm.intercept_)
print(pd.DataFrame({'Predictor': x.columns, 'coefficient':used_cars_lm.coef_}))

intercept 10774.15331010453
           Predictor  coefficient
0          Age_08_04 -2098.271609
1    Automatic_airco   716.916447
2                 HP   528.776428
3                 KM  -731.646898
4      Quarterly_Tax   675.551446
5    Powered_Windows   273.890839
6              Doors   211.737141
7        Sport_Model   220.651204
8   Guarantee_Period   219.830453
9          Automatic   159.039067
10         CD_Player   144.695537
11  Fuel_Type_Diesel   610.839141
12  Fuel_Type_Petrol   590.955467
13           Tow_Bar   -92.660809
14     Mfr_Guarantee    70.191718


most important car specifications for predicting the car’s price are :

Age_08_04 -2098.271609
KM  -731.646898
Automatic_airco   716.916447
Fuel_Type_Diesel   610.839141
Fuel_Type_Petrol   590.955467


In [65]:
used_cars_lm_pred = used_cars_lm.predict(valid_x_scaled_new)
result = pd.DataFrame({'Predicted': used_cars_lm_pred, 'Actual': valid_y,'Residual': valid_y - used_cars_lm_pred})
# print performance measures (validation set)
print("\nModel performance on validation data:")
regressionSummary(valid_y, used_cars_lm_pred)


Model performance on validation data:

Regression statistics

                      Mean Error (ME) : -108.2090
       Root Mean Squared Error (RMSE) : 1220.0170
            Mean Absolute Error (MAE) : 947.2445
          Mean Percentage Error (MPE) : -1.6244
Mean Absolute Percentage Error (MAPE) : 9.6390


In [67]:
r2_all = r2_score(valid_y, used_cars_lm_pred)
print("R-squared value",r2_all)

R-squared value 0.8785231624285312
